In [ ]:
#!/usr/bin/python3
# coding: utf-8
# MHLW

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as td
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, show_and_save_plotly, \
    make_exp_fit_graph
from scipy.optimize import curve_fit
from scipy.special import factorial
from fftdenoise import fft_denoise

In [ ]:
if dt.now().weekday() != 5:
    print("Today is not Saturday.")
    if not "ipy" in sys.argv[0]:
        sys.exit()

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')

In [ ]:
# 厚労省の OpenData を参照する
base_uri = "https://www.mhlw.go.jp/content/"

raws = dict(
    posis = "pcr_positive_daily.csv",
    # 日別PCR検査人数よりも検査機関別の数値を使用すべき
    tests = "pcr_tested_daily.csv",
    cases = "cases_total.csv",
    recov = "recovery_total.csv",
    death = "death_total.csv",
    pcr = "pcr_case_daily.csv")

offsets = dict(
    dates = 0, # 日付オフセット
    cases = 1, # 入院治療を要する者(Total)
    death = 2, # 死亡者数(Total)
    pcr   = 3, # PCR検査件数 3:感染研 4:検疫 5:保健所 6:民間 7:大学 8:医療機関 9:民間自費
    pcrs  = 10, # 上記の合算
    posis = 11, # 陽性者数(Daily)
    tests = 12, # PCR検査 人数(Daily)
    recov = 13, # 退院（Total)
    ratio = 14, # 陽性率(Daily) = 陽性者数 / 検査人数
    total = 15, # 陽性者数(Total)
    ) # 

# 集計期間
dt_range = (dt.today() - dt.strptime(DT_OFFSET, "%Y/%m/%d")).days
# 配列初期化
all_data_arr = []
for i in np.arange(dt_range):
    all_data_arr.append([i, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# データの取得
for k, v in raws.items():
    download_if_needed(base_uri, v)
# データの集計
for k, v in raws.items():
    if v != 0:
        csv2array(all_data_arr, k, v, offsets[k])
# 陽性率等の計算
for i in np.arange(dt_range):
    div = all_data_arr[i][offsets['pcrs']]
    if div != 0:
        all_data_arr[i][offsets['ratio']] = max(0, min(100, (all_data_arr[i][offsets['posis']] / div) * 100))
    if i == 0:
        all_data_arr[i][offsets['total']] = all_data_arr[i][offsets['posis']]
    else:
        all_data_arr[i][offsets['total']] = all_data_arr[i][offsets['posis']] + all_data_arr[i-1][offsets['total']]

all_data_np = np.array(all_data_arr)

In [ ]:
#for line in all_data_arr:
#    print(line)

In [ ]:
updated = (dt.strptime(DT_OFFSET, "%Y/%m/%d") + td(days=int(all_data_np[-1][0]))).isoformat()[:10]
with open("mhlw.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
print("updated: {}, prev: {}".format(updated, prev))
if prev == updated:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

with open("mhlw.prev.tmp", "wt") as f:
    f.write(updated)

In [ ]:
from_date = dt.strptime(DT_OFFSET, "%Y/%m/%d")
xbins = [from_date + td(days=i) for i in range(dt_range)]
days2pred = 4 * 7
xbins_pred = [from_date + td(days=i) for i in range(dt_range + days2pred)]

In [ ]:
ave_mov_days = 7
# 移動平均を算出する
posis_mov_mean = moving_average(all_data_np[:, offsets['posis']])
ratio_mov_mean = moving_average(all_data_np[:, offsets['ratio']])
print("陽性者数(移動平均): {}".format(posis_mov_mean[-1]))
print("　陽性率(移動平均): {}".format(ratio_mov_mean[-1]))

In [ ]:
#all_data_np

In [ ]:
X = np.arange(0, len(posis_mov_mean))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]
y_posis = get_gpr_predict(X, all_data_np[:, offsets['posis']], X_pred, 80, 10, 200)
y_ratio = get_gpr_predict(X, all_data_np[:, offsets['ratio']], X_pred, 80, 10, 200)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['posis']], mode='markers', name='陽性者数', marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_posis, mode='lines', name='予測値', line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=posis_mov_mean, name='移動平均', opacity=0.5), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['ratio']], mode='markers', name='陽性率[%]', marker=dict(size=4)), secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ratio, mode='lines', name='予測値', line=dict(width=1)), secondary_y=True)
fig.add_trace(go.Bar(x=xbins, y=ratio_mov_mean, name='移動平均', opacity=0.8, marker_color='yellow'), secondary_y=True)
fig.update_layout(
    barmode='overlay',
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[30], xbins_pred[-1]]),
    yaxis=dict(title='人数', type='log'),
    yaxis2=dict(title='陽性率[%]', range=[0,50]),
    title='全国 新型コロナ 陽性者数/陽性率 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-posis.jpg", js=False)

In [ ]:
#fft_denoise(xbins[200:], all_data_np[200:, offsets['posis']], freq_int=0.15, freq_th=0.07, freq_min_A=0.01)

In [ ]:
y_tests = get_gpr_predict(X, all_data_np[:, offsets['pcrs']], X_pred, 1, 1, 5)

In [ ]:
# 移動平均を算出する
tests_mov_mean = moving_average(all_data_np[:, offsets['pcrs']])
print("検査人数(移動平均): {}".format(tests_mov_mean[-1]))

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['pcrs']], mode='markers', name='検査人数', marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_tests, mode='lines', name='予測値', line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=tests_mov_mean, name='移動平均', opacity=0.5), secondary_y=False)
fig.add_trace(go.Scatter(x=xbins, y=all_data_np[:, offsets['ratio']], mode='markers', name='陽性率[%]', marker=dict(size=4)), secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ratio, mode='lines', name='予測値', line=dict(width=1)), secondary_y=True)
fig.add_trace(go.Bar(x=xbins, y=ratio_mov_mean, name='移動平均', opacity=0.8, marker_color='yellow'), secondary_y=True)
fig.update_layout(
    barmode='overlay',   
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[30], xbins_pred[-1]]),
    yaxis=dict(title='人数'),#, range=[0, np.max(y_tests)]),
    yaxis2=dict(title='陽性率[%]', range=[0,50]),
    title='全国 新型コロナ 検査人数/陽性率 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-tests.jpg", js=False)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['total']], 
                     name='陽性者', opacity=0.8, marker_color='#c08080'), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['recov']], 
                     name='退院者', opacity=0.8, marker_color='#00c000'), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=all_data_np[:, offsets['cases']], 
                     name='入院中', opacity=0.8, marker_color='yellow'), secondary_y=False)
deads = all_data_np[:, offsets['death']]
deads_os = 0
for i in range(7):
    if deads[-(1+i)] == 0:
        deads_os = i + 1
print("deads offset: {}".format(deads_os))
if deads_os != 0:
    fig.add_trace(go.Scatter(x=xbins[:-deads_os], y=deads[:-deads_os], name="死者",
                             line=dict(width=1, color='magenta')), secondary_y=True)    
else:
    fig.add_trace(go.Scatter(x=xbins, y=deads, name="死者",
                             line=dict(width=1, color='magenta')), secondary_y=True)
fig.update_layout(
    barmode='overlay',
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
              range=[xbins[40], xbins[-1]]),
    yaxis=dict(title='人数'),
    yaxis2=dict(range=[0, np.max(all_data_np[:, offsets['death']])+10]),
    title='全国 新型コロナ 陽性者/退院者/入院中/死者 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-total.jpg", js=False)

In [ ]:
tw_body_total = "全国 新型コロナ 累計陽性者/退院者/死者(" + today_str + ") "
tw_body_total += " https://geneasyura.github.io/cov19-hm/mhlw.html "
tw_body_tests = "全国 新型コロナ 検査人数/陽性率(" + today_str + ") "
tw_body_tests += " https://geneasyura.github.io/cov19-hm/mhlw.html "
tw_body_posis = "全国 新型コロナ 陽性者/陽性率(" + today_str + ") "
tw_body_posis += " https://geneasyura.github.io/cov19-hm/mhlw.html "

In [ ]:
tw = get_twitter()
tweet_with_image(tw, "docs/images/mhlw-posis.jpg", tw_body_posis)
tweet_with_image(tw, "docs/images/mhlw-tests.jpg", tw_body_tests)
tweet_with_image(tw, "docs/images/mhlw-total.jpg", tw_body_total)

In [ ]:
# 実効再生産数
ogiwara_uri = "https://toyokeizai.net/sp/visual/tko/covid19/csv/"
ern_file = "effective_reproduction_number.csv"
download_if_needed(ogiwara_uri, ern_file)

In [ ]:
ern_data_arr = []
for i in np.arange(dt_range):
    ern_data_arr.append([i, 0, 0, 0])
csv2array(ern_data_arr, 'ern', ern_file, 1)
ern_data_np = np.array(ern_data_arr)
#print(ern_data_np[:,1])

In [ ]:
y_ern = get_gpr_predict(X, ern_data_np[:, 1], X_pred, 80, 10, 200)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=xbins, y=ern_data_np[:, 1], name="実効再生産数", opacity=0.5))
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ern, mode='lines', name='予測値', line=dict(width=1)))
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins[44], xbins_pred[-1]]),
    yaxis=dict(title='実効再生産'),
    title='全国 新型コロナ 実効再生産数 ({})'.format(today_str),
)
show_and_save_plotly(fig, "ogiwara-ern.jpg", js=False)

In [ ]:
tw_body_ern = "全国 新型コロナ 実効再生産数 ({})".format(today_str)
tw_body_ern += " https://geneasyura.github.io/cov19-hm/mhlw.html "
tweet_with_image(tw, "docs/images/ogiwara-ern.jpg", tw_body_ern)

In [ ]:
title = '全国 新型コロナ 新規陽性者移動平均/指数近似 (' + today_str + ')'
xdata = np.array(xbins)
#ydata = all_data_np[:, offsets['posis']]
ydata = posis_mov_mean
xos = 310
make_exp_fit_graph(tw,
    xdata[xos:], ydata[xos:],
    title, "mhlw-fit.jpg",
    "mhlw-doubling-time.html", "mhlw.html", needs_tw=False)

In [ ]:
np_influ = np.loadtxt("csv/influ.csv", skiprows=1, delimiter=',', dtype=float)

In [ ]:
np_influ

In [ ]:
# 0-max normalization 
for j in np.arange(1, np_influ.shape[1]):
    peak = 1#np_influ[:, j].max()
    np_influ[:, j] = np_influ[:, j] / peak

In [ ]:
mena_influ = np_influ[:, 1:].mean(axis=1)

In [ ]:
# W36 (2020/8/31) から
os_w36 = (dt(2020, 8, 31) - from_date).days
#print(from_date, w36of2020, os_w36)
xbins = all_data_np[os_w36:, offsets['dates']]
ybins = all_data_np[os_w36:, offsets['posis']]
y_by_week = []
w_by_week = []
for i in np.arange(0, len(xbins), 7):
    w_by_week.append(int(36 + (i / 7)))
    y_by_week.append(int(ybins[i:i + 7].sum()))
print(y_by_week)
print(w_by_week)

In [ ]:
def poisson_func(x, a, b):
    return b * ((a**x * np.exp(-a)) / factorial(x))

In [ ]:
y_values = np.array(y_by_week) - min(y_by_week)
y_values[:4] = 0
(a, b), p0 = curve_fit(
    poisson_func, np.arange(len(w_by_week)), y_values, maxfev=1000
    )
print("a:{}, b:{}, p0:{}".format(a, b, p0))

In [ ]:
xhat = np.arange(40)# + w_by_week[0]
yhat = poisson_func(xhat, a, b) + min(y_by_week)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=w_by_week, y=y_by_week,
    mode='lines+markers', name='COVID-19',
    line=dict(width=.5),
    marker=dict(size=5)), secondary_y=False)
#fig.add_trace(go.Scatter(
#    x=xhat+w_by_week[0], y=yhat,
#    mode='lines', name='ポワソン分布予測',
#    line=dict(width=.5)),
#    secondary_y=False)
fig.add_trace(go.Scatter(
    x=np_influ[:, 0], y=mena_influ, 
    line=dict(width=.7),
    name='インフル平均', opacity=0.5), secondary_y=True)
for j in np.arange(1, np_influ.shape[1]):
    fig.add_trace(go.Scatter(
        x=np_influ[:, 0], y=np_influ[:, j], 
        line=dict(width=.7),
        name='インフル{}'.format(2020-j), opacity=0.5), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='週数'),
    yaxis=dict(title='COVID-19 感染者数'),
    yaxis2=dict(title='インフルエンザ 感染者数'),
    title='全国 新型コロナ インフルとの比較 ({})'.format(today_str),
)
show_and_save_plotly(fig, "mhlw-influ.jpg", js=False)

In [ ]:
tw_body_influ = "全国 新型コロナ インフルとの比較 ({})".format(today_str)
tw_body_influ += " https://geneasyura.github.io/cov19-hm/influ.html "
tweet_with_image(tw, "docs/images/mhlw-influ.jpg", tw_body_influ)